In [40]:
import sys
import os
import numpy as np
from collections import Counter
from collections import defaultdict
import json
from nltk.metrics import distance

In [41]:
def read_source(directory_path):
    source_arr = []
    for root, dirs, files in os.walk(directory_path):
        for f in files:
            with open(directory_path + "/" + f, "r") as source:
                s = source.read()
                source_arr.append((f, f.split('_')[0], s))
    return source_arr

In [42]:
def get_file_extensions():
    path = os.path.expanduser("~/Dropbox/classes/Fall 2017/project/data/{}/source/".format("AAN"))
    arr = read_source(path)
    print Counter([a[0].split('.')[-1] for a in arr])
    
get_file_extensions()

Counter({'py': 10138, 'java': 4974, 'md': 1435, 'cpp': 902, 'c': 626, 'js': 376, 'pl': 73, 'r': 4})


In [43]:
def read_json_file():
    path = os.path.expanduser("~/Dropbox/classes/Fall 2017/project/data/AAN/source_features.json")
    with open(path, "r") as f:
        file_list = json.loads(f.read())
    
    return file_list
    

In [51]:
def search_for_name(keyword, json_dict, type_filter = None):
    matches = []
    for file_name in json_dict.keys():
        for line in json_dict[file_name]:
            right_type = (type_filter is None or line["type"] == type_filter)
            if line["name"] == keyword and right_type:
                matches.append((file_name, line))
    return matches
    
    

In [67]:
def search_for_docstring(keyword, json_dict, type_filter = None):
    matches = []
    for file_name in json_dict.keys():
        for line in json_dict[file_name]:
            right_type = (type_filter is None or line["type"] == type_filter)
            if keyword in line["docstring"] and right_type:
                matches.append((file_name, line))
    return matches

In [45]:
def read_source(file_name):
    try:
        with open(file_name, "r") as f:
            source = f.read()
        return source
    except Exception:
        return ""
    

In [46]:
def print_source(file_name, line_start, line_end):
    source = read_source(file_name)
    return "\n".join(source.split("\n")[line_start - 1 :line_end])
        

In [47]:
def get_indentation(line_string):
    whitespace = [" ", "\t"]
    indentation = ""
    for char in line_string:
        if char in whitespace:
            indentation += char
        else:
            break
    return indentation

In [48]:
def infer_end_line(file_name, line_dict):
    source = read_source(file_name)
    lines = source.split("\n")
    
    target_indentation = get_indentation(lines[line_dict["line"] - 1])
    i = line_dict["line"]
    while i < len(lines):
        line = lines[i]
        if (get_indentation(line) == target_indentation) and len(line) > 0:
            if i == line_dict["line"]:
                return i
            return i - 1
        i += 1
    return i
        
    
    

In [56]:
def find_matches_by_name(json_dict, search_term, type_filter=None):
    matches = search_for_name(search_term, json_dict, type_filter)
    if len(matches):
        end =  infer_end_line(matches[0][0], matches[0][1])
        return print_source(matches[0][0], matches[0][1]["line"], end)
    else:
        return "No Matches Found."

    

In [68]:
def find_matches_by_docstring(json_dict, search_term, type_filter=None):
    matches = search_for_docstring(search_term, json_dict, type_filter)
    if len(matches):
        end =  infer_end_line(matches[0][0], matches[0][1])
        return print_source(matches[0][0], matches[0][1]["line"], end)
    else:
        return "No Matches Found."

In [57]:
json_dict = read_json_file()

In [74]:
# print find_matches_by_name(json_dict, "sofax", "function")
print find_matches_by_docstring(json_dict, "Relu")

class ReluLayer(Layer):
  def __init__(self, *args, **kwargs):
    super(ReluLayer, self).__init__(*args, **kwargs)

  @classmethod
  def IsLayerType(cls, proto):
    return proto.hyperparams.activation == deepnet_pb2.Hyperparams.RECTIFIED_LINEAR

  def ApplyActivation(self, neg=False):
    if neg:
      state = self.neg_state
    else:
      state = self.state
    state.lower_bound(0)

  def Sample(self, neg=False):
    if neg:
      sample = self.neg_sample
      state = self.neg_state
    else:
      sample = self.sample
      state = self.state
    state.sample_gaussian(target=sample, mult=1.0)
    sample.lower_bound(0)

  def ComputeDeriv(self):
    """Compute derivative w.r.t input given derivative w.r.t output."""
    self.deriv.apply_rectified_linear_deriv(self.state)

  def GetLoss(self, get_deriv=False, acc_deriv=False, **kwargs):
    """Compute loss and also deriv w.r.t to it if asked for.

    Compute the loss function. Targets should be in self.data, predictions
    should